In [ ]:
import spacy
import pandas as pd
import numpy as np


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
df = pd.read_excel("usecase_training_data.xlsx",sheet_name='final_sheet')


In [ ]:
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 13:03:44.488301: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:

training_data = []
for index,row in df.iterrows():
  temp_dict = {}
  temp_dict['text'] = row['snippet']
  temp_dict['entities'] = []
  start = row['start']
  end = row['end']
  label = row['label'].upper()
  temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

{'text': 'By Jonathan Sherwood | Sep 22, 2022 | Press Release | 0 comments\n\nGlobal Bioenergies today announced that it has successfully completed the REWOFUEL project – and a key part of this project was Sweetwater’s Sunburst technology, which deconstructed wood into cellulose and hemicellulose, allowing Global Bioenergies to convert them into sustainable high-performance components for road and aviation fuels.\nOne of Sweetwater’s partners, Fibenol, used Sunburst to help generate sugars from wood, which Global Bioenergies then produced bio-isobutane, which is a key ingredient in sustainable aviation fuels.', 'entities': [(390, 404, 'Y')]}


In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.load("en_core_web_lg") # load a new spacy model
doc_bin = DocBin()

In [ ]:

from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

 36%|███▌      | 140/390 [00:00<00:00, 488.80it/s]

Skipping entity


100%|██████████| 390/390 [00:00<00:00, 596.19it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 13:04:39.619354: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 13:04:49.404169: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-01-23 13:04:50,461] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-01-23 13:04:50,481] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2023-01-23 13:04:50,493] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2023-01-23 13:04:53,048] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x389ee000 @  0x7f11a777f680 0x7f11a779fbdd 0x7f119e94ce09 0x7f119e94bcdf 0x7f119e948675 0x7f119e948e2e 0x504866 0x56bbe1 0x569d8a 0x5f60c

In [ ]:
nlp_ner = spacy.load("model-best")


In [ ]:
doc = nlp_ner("In a conservative market, composite products are becoming increasingly popular in construction projects. They are replacing traditional materials used in constructions exposed to harsh environments (such as metal) and offer significantly improved strength, corrosion resistance and durability. Our Elium® thermoplastic resin is used for thermoplastic composites that are used in the main structure of buildings, replacing steel frames. These composite rebars offer excellent corrosion resistance. Elium® resin can also be used to make façade panels on the exterior of buildings, as well as sanitary furniture (worktops, washbasins).")

In [ ]:
for ent in doc.ents:

  print(ent.text,"---",ent.label_)

to make façade panels on the exterior of buildings --- N


In [ ]:
df_ = pd.read_excel('/content/model_training.xlsx',sheet_name='model_test')
print(df_)

     usecase_id                     usecase  \
0          3281   building and construction   
1          3280           Textiles & Fibers   
2          3279            Sports & Leisure   
3          3278        industrial materials   
4          3277  Compression molded sheets.   
..          ...                         ...   
391        1727       offshore applications   
392        1726      Electricity Generation   
393        1725      Wastewater Remediation   
394        1724          Water Desalination   
395        1723                 Aquaculture   

                                               snippet  
0    In a conservative market, composite products a...  
1    Specially developed from a formula with silver...  
2    Specially developed from a formula with silver...  
3    Mitsubishi Chemical Corporation has developed ...  
4    Braskem UTEC is a line of ultra molecular weig...  
..                                                 ...  
391  Archimedes Waveswing is a uniqu

In [ ]:
df_['output'] = None

df_.drop(lst,axis= 0,inplace = True)


for index,row in df_.iterrows():

  snippet = row['snippet']

  print(snippet)

  lst = []

  doc = nlp_ner(snippet)

  for i in doc.ents:

    lst.append({'usecase':i.text,'label':i.label_})

  df_.at[index,'output'] = lst

df_.to_excel('output_excel.xlsx')
print('saved')


In a conservative market, composite products are becoming increasingly popular in construction projects. They are replacing traditional materials used in constructions exposed to harsh environments (such as metal) and offer significantly improved strength, corrosion resistance and durability. Our Elium® thermoplastic resin is used for thermoplastic composites that are used in the main structure of buildings, replacing steel frames. These composite rebars offer excellent corrosion resistance. Elium® resin can also be used to make façade panels on the exterior of buildings, as well as sanitary furniture (worktops, washbasins).
Specially developed from a formula with silver ions, Amni® Biotech Eco yarn controls the proliferation of bacteria that cause unwanted odor, allowing for greater comfort and a feeling of freshness during daily activities. This yarn quickly decomposes after being disposed in landfills.
Specially developed from a formula with silver ions, Amni® Biotech Eco yarn contr